# Moving Data into Bronze Layer from Landing

In [17]:
from pyspark.sql import functions as F
from datetime import datetime

PROCESSING_START_TIME = datetime.now()

StatementMeta(, 299edc03-3562-4dab-bb17-6d926a6df80a, 19, Finished, Available, Finished)

In [18]:
# Toggle parameter

LANDING_PATH = "abfss://fabric_dev@onelake.dfs.fabric.microsoft.com/fabric_LH_sales.Lakehouse/Files/Landing"
BRONZE_PATH  = "abfss://fabric_dev@onelake.dfs.fabric.microsoft.com/fabric_LH_sales.Lakehouse/Tables/Bronze"
BRONZE_TABLE = "dbo.stsales_bronze"
RUN_LANDING_TO_BRONZE = True

PROCESSING_START_TIME = datetime.now()
PROCESSING_YEAR = PROCESSING_START_TIME.year
PROCESSING_MONTH = PROCESSING_START_TIME.month

print(f" Parameters Loaded — Year: {PROCESSING_YEAR}, Month: {PROCESSING_MONTH}")
print(f" Source: {LANDING_PATH}")
print(f" Target: {BRONZE_PATH}")

StatementMeta(, 299edc03-3562-4dab-bb17-6d926a6df80a, 20, Finished, Available, Finished)

 Parameters Loaded — Year: 2025, Month: 10
 Source: abfss://fabric_dev@onelake.dfs.fabric.microsoft.com/fabric_LH_sales.Lakehouse/Files/Landing
 Target: abfss://fabric_dev@onelake.dfs.fabric.microsoft.com/fabric_LH_sales.Lakehouse/Tables/Bronze


#### Load data from Landing

In [21]:
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, LongType, TimestampType

if RUN_LANDING_TO_BRONZE:
    print("Reading data from Landing...")

    # Define schema with timestamps as STRING first to avoid BINARY -> TIMESTAMP issue
    landing_schema = StructType([
        StructField("Row_ID", LongType(), True),
        StructField("Order_ID", StringType(), True),
        StructField("Order_Date", StringType(), True),  # read as string first
        StructField("Ship_Date", StringType(), True),   # read as string first
        StructField("Ship_Mode", StringType(), True),
        StructField("Customer_ID", StringType(), True),
        StructField("Customer_Name", StringType(), True),
        StructField("Segment", StringType(), True),
        StructField("Country", StringType(), True),
        StructField("City", StringType(), True),
        StructField("State", StringType(), True),
        StructField("Postal_Code", LongType(), True),
        StructField("Region", StringType(), True),
        StructField("Product_ID", StringType(), True),
        StructField("Category", StringType(), True),
        StructField("Sub_Category", StringType(), True),
        StructField("Product_Name", StringType(), True),
        StructField("Sales", DoubleType(), True),
        StructField("Quantity", LongType(), True),
        StructField("Discount", DoubleType(), True),
        StructField("Profit", DoubleType(), True),
        StructField("Processing_Time", TimestampType(), True)
    ])

    # Read Landing data
    bronze_df = spark.read.schema(landing_schema).parquet(LANDING_PATH)

    # Safely convert string columns to timestamp
    bronze_df = bronze_df.withColumn("Order_Date", F.to_timestamp("Order_Date", "dd-MM-yyyy"))
    bronze_df = bronze_df.withColumn("Ship_Date", F.to_timestamp("Ship_Date", "dd-MM-yyyy"))

    row_count = bronze_df.count()
    if row_count == 0:
        print("No new data in Landing, skipping Bronze load.")
    else:
        # Add Year, Month, ProcessingTime
        bronze_df = (
            bronze_df
            .withColumn("Order_Date_parsed", F.to_date("Order_Date", "dd-MM-yyyy"))
            .withColumn("Year", F.year("Order_Date_parsed"))
            .withColumn("Month", F.month("Order_Date_parsed"))
            .withColumn("Landing_to_Bronze_ProcessingTime", F.current_timestamp())
        )

        # Write to Bronze folder (append, partitioned)
        (
            bronze_df.write
            .mode("append")
            .option("mergeSchema", "true")
            .partitionBy("Year", "Month")
            .parquet(BRONZE_PATH)
        )

        print(f"Successfully loaded {row_count:,} rows into Bronze folder: {BRONZE_PATH}")

        # Register temporary view for downstream pipelines
        bronze_df.createOrReplaceTempView("bronze_temp_view")
        print("Temporary view 'bronze_temp_view' created for downstream processing.")

StatementMeta(, 299edc03-3562-4dab-bb17-6d926a6df80a, 23, Finished, Available, Finished)

Reading data from Landing...


Py4JJavaError: An error occurred while calling o5376.parquet.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 32.0 failed 4 times, most recent failure: Lost task 1.3 in stage 32.0 (TID 212) (vm-96f28098 executor 2): org.apache.spark.SparkException: Parquet column cannot be converted in file abfss://fabric_dev@onelake.dfs.fabric.microsoft.com/fabric_LH_sales.Lakehouse/Files/Landing/Year=2025/Month=10/part-00007-ca9a2c2c-8947-4e9a-b542-4010b8d9a2d3.c000.snappy.parquet. Column: [Order_Date], Expected: string, Found: INT64.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.unsupportedSchemaColumnConvertError(QueryExecutionErrors.scala:856)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextIterator(FileScanRDD.scala:346)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.hasNext(FileScanRDD.scala:175)
	at org.apache.spark.sql.execution.FileSourceScanExec$$anon$1.hasNext(DataSourceScanExec.scala:634)
	at org.apache

In [ ]:
        # LOG PIPELINE RUN
        PROCESSING_END_TIME = datetime.now()

        duration_seconds = (PROCESSING_END_TIME - PROCESSING_START_TIME).total_seconds()
        duration_minutes = round(duration_seconds / 60, 2)

        log_data = [("BROWN_LAYER_DATASET", PROCESSING_START_TIME, PROCESSING_END_TIME, duration_minutes, "Landing -> Bronze", BRONZE_PATH, row_count)]
        log_df = spark.createDataFrame(log_data, ["Dataset", "Start_Timestamp", "End_Timestamp", "run_duration", "Stage", "Destination", "Row_Count"])
        log_df.write \
        .format("delta") \
        .mode("append") \
        .option("mergeSchema", "true") \
        .saveAsTable("dbo.pipeline_log")

        print(" Pipeline load logged successfully.")



print(" Landing → Bronze completed successfully.")

StatementMeta(, 299edc03-3562-4dab-bb17-6d926a6df80a, -1, Cancelled, , Cancelled)